In [ ]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.single_ch_transmitter_task import SingleChannelTransmitterTask
from rfsoc_rfdc.single_ch_receiver_task import SingleChannelReceiverTask

from rfsoc_rfdc.multi_ch_transmitter_task import MultiChannelTransmitterTask
from rfsoc_rfdc.multi_ch_receiver_task import MultiChannelSingleChannelReceiverTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask

ol = RFSoCOverlay()

In [ ]:
import sys
import os

def validate_arguments(waveform_path, dac_samp_rate, adc_samp_rate, carrier_freq):
    if not os.path.exists(waveform_path):
        raise FileNotFoundError(f"The specified waveform file '{waveform_path}' does not exist.")
    try:
        dac_samp_rate, adc_samp_rate = int(dac_samp_rate), int(adc_samp_rate)
        carrier_freq = int(carrier_freq)
    except ValueError:
        raise ValueError("DAC/ADC sampling rate and carrier frequency must be integers.")
    if dac_samp_rate < 500e6 or dac_samp_rate > 10e9:
        raise ValueError("DAC sampling rate ranges from 500MHz to 10GHz.")
    if adc_samp_rate < 500e6 or adc_samp_rate > 2.5e9:
        raise ValueError("DAC sampling rate ranges from 500MHz to 2.5GHz.")
    return True

matlab_waveform = "./wave_files/Tx.mat"
dac_samp_rate, adc_samp_rate = 2e9, 2e9
carrier_freq = 0

try:
    if validate_arguments(matlab_waveform, dac_samp_rate, adc_samp_rate, carrier_freq):
        print("Arguments validated successfully.")
except Exception as e:
    print(f"Error: {e}")
    sys.exit(1)

rfdc_t = RfdcTask(ol, dac_samp_rate, adc_samp_rate, carrier_freq)
# cmac_t = CmacTask(ol)
led_t = BlinkLedTask(ol)
tx_t = SingleChannelTransmitterTask(ol, matlab_waveform)
# tx_t = MultiChannelTransmitterTask(ol)
rx_t = SingleChannelReceiverTask(ol)
# rx_t = MultiChannelSingleChannelReceiverTask(ol)

for task in [rfdc_t]:
    task.start()
    task.join()

parallel_task = [led_t, tx_t, rx_t]
for task in parallel_task:
    task.start()
for task in parallel_task:
    task.join()